In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras import Model

## Load and prepare MNIST dataset

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Normalizing inputs between 0-1
x_train, x_test =  x_train/255., x_test/255.  

# Add channel dimension
x_train = x_train[..., tf.newaxis].astype('float32')
x_test = x_test[..., tf.newaxis].astype('float32')

In [3]:
# training parameters
batch_sz = 32
num_epochs = 5

## Shuffle (only training-set) & batch the data

In [4]:
train_ds = tf.data.Dataset.from_tensor_slices(
            (x_train, y_train)).shuffle(10000).batch(batch_sz)

test_ds = tf.data.Dataset.from_tensor_slices(
            (x_test, y_test)).batch(batch_sz)

## Build CNN-based classification model

In [5]:
class SimpleCNNModel(Model):
    def __init__(self):
        super(SimpleCNNModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten1 = Flatten()
        self.dense1 = Dense(128, activation='relu')
        self.dense2 = Dense(10)
        
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten1(x)
        x = self.dense1(x)
        x = self.dense2(x)
        
        return x

In [6]:
cnnmodel = SimpleCNNModel()  # create a model instance

## Define loss and optimizer

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

## Performance metric

In [8]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## Training model step

In [9]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:  # need to understand gradient_tape
        predictions = cnnmodel(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, cnnmodel.trainable_variables)
    optimizer.apply_gradients(zip(gradients, cnnmodel.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

## Test model step

In [10]:
@tf.function
def test_step(images, labels):
    predictions = cnnmodel(images, training=False)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [11]:
for epoch in range(num_epochs):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
        
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
        
    print(
        f'Training loss: {round(float(train_loss.result()), 3)}, '
        f'Training accuracy: {round(float(train_accuracy.result() * 100.), 3)}, '
        f'Test loss: {round(float(test_loss.result()), 3)}, '
        f'Test accuracy: {round(float(test_accuracy.result() * 100.), 3)}'
    )

Training accuracy: 95.905, Test accuracy: 97.79
Training accuracy: 98.618, Test accuracy: 98.14
Training accuracy: 99.255, Test accuracy: 98.3
Training accuracy: 99.505, Test accuracy: 98.47
Training accuracy: 99.667, Test accuracy: 98.38
